In [2]:
#Linear Regression

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
inputs = np.load("data/regression/inputs.npy")

In [4]:
labels = np.load("data/regression/labels.npy")


In [5]:
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)

In [6]:
linearReg = LinearRegression().fit(X_train, y_train)

In [7]:
y_pred = linearReg.predict(X_test)

In [8]:
from sklearn.metrics import r2_score
r2_score(y_pred, y_test)

0.9125921264582036

In [9]:
#Logistic Regression

#from sklearn.linear_model import LogisticRegression

#logisticReg = LogisticRegression(random_state=0).fit(X_train, y_train)

#Since logistic regression accepts categorical values or integers as labels, therefore it does not fit on the purpose of our dataset because it is continous.



In [23]:
#Ridge Regression
from sklearn.linear_model import Ridge

ridgeReg = Ridge(alpha=1.0)

ridgeRegFit = ridgeReg.fit(X_train, y_train)

In [24]:
y_pred_ridge = ridgeRegFit.predict(X_test)


In [25]:
r2_score(y_pred_ridge, y_test)

0.9129271221104657

In [14]:
from sklearn.model_selection import cross_val_score

cross_val_score(ridgeRegFit, inputs, labels)

array([0.88137519, 0.88583137, 0.89018479, 0.92147377, 0.88438868])

In [26]:
import pandas as pd
from sklearn import metrics


def test(models, data, iterations = 1000):
    results = {}
    for i in models:
        r2_train = []
        r2_test = []
        for j in range(iterations):
            X_train, X_test, y_train, y_test = train_test_split(data['X'],
                                                                data['Y'],
                                                                test_size= 0.2)
            r2_test.append(metrics.r2_score(y_test,
                                            models[i].fit(X_train,
                                                         y_train).predict(X_test)))
            r2_train.append(metrics.r2_score(y_train,
                                             models[i].fit(X_train,
                                                          y_train).predict(X_train)))
        results[i] = [np.mean(r2_train), np.mean(r2_test)]
    return pd.DataFrame(results)

In [27]:
from sklearn.linear_model import Lasso

models = {'OLS': LinearRegression(),
         'Lasso': Lasso(),
         'Ridge': Ridge(),}

test(models, data={'X': inputs, 'Y':labels})

/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.048e+02, tolerance: 1.688e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.048e+02, tolerance: 1.688e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

,OLS,Lasso,Ridge
0,0.898760,0.897618,0.898599
1,0.891487,0.892966,0.892101


In [28]:
from sklearn.model_selection import GridSearchCV

lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

models2 = {'OLS': LinearRegression(),
           'Lasso': GridSearchCV(Lasso(),
                               param_grid=lasso_params).fit(inputs, labels).best_estimator_,
           'Ridge': GridSearchCV(Ridge(),
                               param_grid=ridge_params).fit(inputs, labels).best_estimator_,}


/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.237e+04, tolerance: 1.704e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.724e+04, tolerance: 1.699e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

In [29]:
from sklearn.linear_model import Lasso

models = {'OLS': LinearRegression(),
         'Lasso': Lasso(),
         'Ridge': Ridge(),}

test(models2, data={'X': inputs, 'Y':labels})

/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.140e+04, tolerance: 1.713e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.140e+04, tolerance: 1.713e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

,OLS,Lasso,Ridge
0,0.898500,0.898203,0.897436
1,0.892438,0.893455,0.892860


In [30]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

lasso_params = {'fit__alpha':[0.005, 0.02, 0.03, 0.05, 0.06]}
ridge_params = {'fit__alpha':[550, 580, 600, 620, 650]}

pipe1 = Pipeline([('poly', PolynomialFeatures()),
                 ('fit', LinearRegression())])
pipe2 = Pipeline([('poly', PolynomialFeatures()),
                 ('fit', Lasso())])
pipe3 = Pipeline([('poly', PolynomialFeatures()),
                 ('fit', Ridge())])

models3 = {'OLS': pipe1,
           'Lasso': GridSearchCV(pipe2,
                                 param_grid=lasso_params).fit(inputs, labels).best_estimator_ ,
           'Ridge': GridSearchCV(pipe3,
                                 param_grid=ridge_params).fit(inputs, labels).best_estimator_,}

/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.016e+04, tolerance: 1.704e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.151e+04, tolerance: 1.699e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

In [31]:
test(models3, data={'X': inputs, 'Y':labels})

/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.856e+04, tolerance: 1.684e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.856e+04, tolerance: 1.684e+02
  model = cd_fast.enet_coordinate_descent(
/Users/longvo/miniforge3/envs/bigdata/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

,OLS,Lasso,Ridge
0,0.923766,0.913958,0.913523
1,0.848566,0.876119,0.875421


In [ ]:
#Lasso
from sklearn import linear_model
lasso = linear_model.Lasso(alpha=0.1, fit_intercept=False, tol=0.001, max_iter=1000, positive=True)
lassoFit = lasso.fit(X_train, y_train)
y_pred_lasso = lassoFit.predict(X_test)
r2_score(y_pred_lasso, y_test)